In [41]:
import tensorflow as tf
import os
from tensorflow import keras
import numpy as np
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from multiprocessing import Process
from IPython.display import clear_output

In [42]:
FRAME_SKIP = 2
FRAME_SIZE = (150,150)

In [43]:
# !git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos

In [44]:
# !mkdir Data
# !mkdir -p ./Data/Video/Violent
# !mkdir -p ./Data/Video/NonViolent
# !cp -a ./A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/violent/cam1/. ./Data/Video/Violent/
# !cp -a ./A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/non-violent/cam1/. ./Data/Video/NonViolent/
# clear_output()
# !mkdir -p ./Data/Training/V
# !mkdir -p ./Data/Training/NV

In [45]:
def extract_frames(video_path, output_path, frame_size=(150, 150), frame_skip=2):
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_skip == 0:
            frame = cv2.resize(frame, frame_size)
            frame_filename = os.path.join(output_path, f"{os.path.splitext(os.path.basename(video_path))[0]}_frame_{count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
        count += 1
    cap.release()

def thread_1():
    for i in range(60):
        video_path = f"./Data/Video/Violent/{i+1}.mp4"
        output_path = "./Data/Training/V"
        print(f"Processing Violent Vid-{i}")
        extract_frames(video_path, output_path, frame_size=FRAME_SIZE, frame_skip=FRAME_SKIP)
    print("Violent Extracted")

def thread_2():
    for i in range(60):
        video_path = f"./Data/Video/NonViolent/{i+1}.mp4"
        output_path = "./Data/Training/NV"
        print(f"Processing NonViolent Vid-{i}")
        extract_frames(video_path, output_path, frame_size=FRAME_SIZE, frame_skip=FRAME_SKIP)
    print("Non-Violent Extracted")

In [46]:
t1 = Process(target=thread_1, args=())
t2 = Process(target=thread_2, args=())

t1.start() 
t2.start()

t1.join()
t2.join()
print("Complete")

Processing Violent Vid-0
Processing NonViolent Vid-0
Processing Violent Vid-1
Processing NonViolent Vid-1
Processing NonViolent Vid-2
Processing Violent Vid-2
Processing NonViolent Vid-3
Processing Violent Vid-3
Processing NonViolent Vid-4
Processing Violent Vid-4
Processing NonViolent Vid-5
Processing Violent Vid-5
Processing NonViolent Vid-6
Processing Violent Vid-6
Processing NonViolent Vid-7
Processing Violent Vid-7
Processing NonViolent Vid-8
Processing NonViolent Vid-9
Processing Violent Vid-8
Processing NonViolent Vid-10
Processing Violent Vid-9
Processing NonViolent Vid-11
Processing Violent Vid-10
Processing NonViolent Vid-12
Processing Violent Vid-11
Processing Violent Vid-12
Processing NonViolent Vid-13
Processing NonViolent Vid-14
Processing Violent Vid-13
Processing Violent Vid-14
Processing NonViolent Vid-15
Processing Violent Vid-15
Processing NonViolent Vid-16
Processing Violent Vid-16
Processing Violent Vid-17
Processing NonViolent Vid-17
Processing Violent Vid-18
Proc

In [47]:
base_dir='./Data'
train_dir=os.path.join(base_dir,'Training')
train_violent_dir =os.path.join(train_dir, 'V' )
train_nonviolent_dir=os.path.join(train_dir,'NV')

In [48]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=40,width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,horizontal_flip=True, fill_mode='nearest')

In [49]:
train_generator = train_datagen.flow_from_directory(train_dir,color_mode="rgb", target_size = FRAME_SIZE,batch_size=20,classes=['NV','V'], class_mode='binary', shuffle=True)

Found 10280 images belonging to 2 classes.


In [50]:
model= tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512, activation='relu'),
       tf.keras.layers.Dense(1,activation ='sigmoid')

])

In [51]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [52]:
model1=model.fit(train_generator,steps_per_epoch=50, epochs=30)

Epoch 1/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 702ms/step - accuracy: 0.5422 - loss: 0.6913
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 710ms/step - accuracy: 0.5446 - loss: 0.6928
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 651ms/step - accuracy: 0.5672 - loss: 0.6920
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 679ms/step - accuracy: 0.5971 - loss: 0.6787
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 643ms/step - accuracy: 0.6448 - loss: 0.6550
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 697ms/step - accuracy: 0.7724 - loss: 0.5309
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 643ms/step - accuracy: 0.8208 - loss: 0.4279
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 689ms/step - accuracy: 0.8130 - loss: 0.4289
Epoch 9/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 688ms/step - accuracy: 0.7451 - loss: 0.5074
Epoch 10/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 661ms/step - accuracy: 0.8011 - loss: 0.4317
Epoch 11/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 187ms/step - accuracy: 0.8255 - loss: 0.4128
Epoch 12/30


2024-06-03 23:14:56.522118: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


50/50 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.8379 - loss: 0.3815
Epoch 13/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 646ms/step - accuracy: 0.7964 - loss: 0.4256
Epoch 14/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 664ms/step - accuracy: 0.8570 - loss: 0.3399
Epoch 15/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 647ms/step - accuracy: 0.7832 - loss: 0.4646
Epoch 16/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 640ms/step - accuracy: 0.8380 - loss: 0.3798
Epoch 17/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 693ms/step - accuracy: 0.8621 - loss: 0.3454
Epoch 18/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 679ms/step - accuracy: 0.8333 - loss: 0.3653
Epoch 19/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 655ms/step - accuracy: 0.8384 - loss: 0.3546
Epoch 20/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 662ms/step - accuracy: 0.8522 - loss: 0.3425
Epoch 21/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 37s 734ms/step - accuracy: 0.8743 - loss: 0.3189
Epoch 22/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 162ms/step - accuracy: 0.8392 - loss: 0.3320 
Epoch 23/30


2024-06-03 23:21:01.011384: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


50/50 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - accuracy: 0.8526 - loss: 0.3591
Epoch 24/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 642ms/step - accuracy: 0.8706 - loss: 0.3182
Epoch 25/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 35s 702ms/step - accuracy: 0.8569 - loss: 0.3305
Epoch 26/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 39s 783ms/step - accuracy: 0.8599 - loss: 0.3413
Epoch 27/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 667ms/step - accuracy: 0.8765 - loss: 0.2829
Epoch 28/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 651ms/step - accuracy: 0.8908 - loss: 0.2859
Epoch 29/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 646ms/step - accuracy: 0.8789 - loss: 0.2840
Epoch 30/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 669ms/step - accuracy: 0.8446 - loss: 0.3591


In [53]:
import time
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1717437385.h5
